In [1]:
import minsearch

In [2]:
import json

In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [8]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp'

In [11]:
q = 'the course has already started, can I still enroll?'

In [12]:
index.fit(documents)

In [16]:
boost = {'question': 3.0, 'section':0.5}

results = index.search(
    query = q,
    filter_dict = {'course': 'data-engineering-zoomcamp'},
    boost_dict = boost,
    num_results = 5
)

In [35]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [19]:
from dotenv import load_dotenv
import os


In [20]:
load_dotenv()

True

In [21]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [23]:
from openai import OpenAI

In [24]:
client = OpenAI()

In [27]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role":"user", "content":q}]
)

In [29]:
response.choices[0].message.content

"Certainly! The possibility of enrolling in a course after it has already started often depends on the institution's policies and the specific course requirements. Here are a few steps you can take:\n\n1. **Check the Enrollment Deadline:** Visit the course or institution's website to see if there is a specified cutoff date for late enrollment.\n\n2. **Contact the Instructor:** Reach out directly to the instructor or course coordinator. Explain your situation and express your interest. Instructors can sometimes make exceptions or provide guidance on how to catch up.\n\n3. **Consult Academic Advising:** Academic advisors can provide detailed information about late enrollment processes and what steps you need to take.\n\n4. **Review the Syllabus:** Check if the course content and workload are manageable if you join late. Some courses may have critical assignments or exams early on that could impact your ability to succeed if missed.\n\n5. **Understand the Impact:** Ensure you understand a

In [45]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't' contain the answer, output NONE


QUESTION:{question}
CONTEXT: {context}
""".strip()

In [43]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [44]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [48]:
prompt = prompt_template.format(question=q, context=context).strip()

In [50]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't' contain the answer, output NONE


QUESTION:the course has already started, can I still enroll?
CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fi

In [51]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role":"user", "content":prompt}]
)

response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [57]:
def search(query):
    boost = {'question': 3.0, 'section':0.5}

    results = index.search(
        query = query,
        filter_dict = {'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )
    
    return results

In [72]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
        Use only the facts from the CONTEXT when answering the QUESTION.
        If the CONTEXT doesn't' contain the answer, output NONE
        
        
        QUESTION:{question}
        CONTEXT: {context}
        """.strip()
    
    context = ""
    
    for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [78]:
def llm(prompt):
    response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role":"user", "content":prompt}]
    )
    return response.choices[0].message.content

In [81]:
query = " How do i run kafka?"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [82]:
rag(query)

'To run Kafka producer/consumer/kstreams/etc in terminal for a Java Kafka project:\n\n1. Navigate to your project directory.\n2. Execute the command:\n   ```bash\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\nReplace `<jar_name>` with the actual name of your JAR file. \n\nIf you are working with Python, ensure you have created and activated a virtual environment and installed the required packages as described for the "Module \'kafka\' not found" issue.'

In [83]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [85]:
from elasticsearch import Elasticsearch

In [86]:
es_client = Elasticsearch('http://localhost:9200')

In [87]:
es_client.info()

ObjectApiResponse({'name': 'bad94fae3936', 'cluster_name': 'docker-cluster', 'cluster_uuid': '1Qt29qX6TGq6HVF6t9qqww', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [88]:
index_setting = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course_questions"

es_client.indices.create(index=index_name, body=index_setting)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [89]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [90]:
from tqdm.auto import tqdm

In [92]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document = doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [93]:
query = 'I just discovered this course, can i still join it?'

In [103]:
def elastic_search(query):

    search_query ={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [103]:
def elastic_search(query):

    search_query ={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [105]:
query = " How do i run kafka?"

def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [106]:
result_docs

"To run Kafka, follow these steps based on the provided context:\n\n1. **For running Java Kafka Producer/Consumer/KStreams in the terminal**, navigate to your project directory and execute:\n   ```sh\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\n2. **If you are working with Python Kafka and encounter module issues**, it's recommended to create and use a virtual environment:\n\n   - Create a virtual environment:\n     ```sh\n     python -m venv env\n     source env/bin/activate\n     ```\n   - Install the required packages:\n     ```sh\n     pip install -r ../requirements.txt\n     ```\n   - Activate the virtual environment each time you need it:\n     ```sh\n     source env/bin/activate\n     ```\n   - To deactivate the virtual environment:\n     ```sh\n     deactivate\n     ```\n   - On Windows, the path to activate the virtual environment is slightly different:\n     ```sh\n     env\\Scripts\\activate\n     ```\n\nIf yo